# MNIST Neural Network

### Imports

In [1]:
import tensorflow as tf
import numpy as np
import math

### Load Data and Create Model

In [2]:
mnist = tf.keras.datasets.mnist
mnist = mnist.load_data()
(x_train,y_train),(x_test,y_test)=mnist
x_train,x_test = tf.keras.utils.normalize(x_train,axis=1),tf.keras.utils.normalize(x_test,axis=1)


#print(np.shape([1,2,3,4]))

#print(np.shape(x_train)) #(60000,28,28)

#feed forward NN

hidden_units = 200

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(hidden_units,activation=tf.nn.relu))
# #model.add(tf.keras.layers.Dropout(0.25))
# model.add(tf.keras.layers.Dense(100,activation=tf.nn.relu))
# #model.add(tf.keras.layers.Dropout(0.25))
# model.add(tf.keras.layers.Dense(60,activation=tf.nn.relu))
# #model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax))

optim = tf.keras.optimizers.Adam(learning_rate=0.01)

#could try sparse
model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=[tf.keras.metrics.RootMeanSquaredError()])

steps_per_epoch = len(x_train)//100
print(y_test[0])

Metal device set to: Apple M2
7


2022-07-22 16:26:12.502279: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-22 16:26:12.502368: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
mnist = tf.keras.datasets.mnist
mnist = mnist.load_data()
(x_train,y_train),(x_test,y_test)=mnist
x_train,x_test = tf.keras.utils.normalize(x_train,axis=1),tf.keras.utils.normalize(x_test,axis=1)

#print(np.shape([1,2,3,4]))

#print(np.shape(x_train)) #(60000,28,28)

#feed forward NN

hidden_units = 200

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(hidden_units,activation=tf.nn.relu))
# #model.add(tf.keras.layers.Dropout(0.25))
# model.add(tf.keras.layers.Dense(100,activation=tf.nn.relu))
# #model.add(tf.keras.layers.Dropout(0.25))
# model.add(tf.keras.layers.Dense(60,activation=tf.nn.relu))
# #model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax))

optim = tf.keras.optimizers.Adam(learning_rate=0.01)

#could try sparse
model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

steps_per_epoch = len(x_train)//100

#### Decaying Learning Rate Function

In [4]:
# lr schedule callback
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * math.pow(0.6,epoch), verbose=True)


In [5]:
x_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

### Train

In [6]:
#tf.keras.models.Sequential().fit()
print(y_train)
print(np.shape(y_train))
model.fit(x_train,y_train,epochs=10,steps_per_epoch = steps_per_epoch)#callbacks=[lr_decay_callback])

[5 0 4 ... 5 6 8]
(60000,)
Epoch 1/10


2022-07-22 16:26:13.002354: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-22 16:26:13.149848: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 2s 4ms/step - loss: 0.2085 - accuracy: 0.9363
Epoch 2/10
600/600 [==============================] - 2s 4ms/step - loss: 0.1055 - accuracy: 0.9669
Epoch 3/10
600/600 [==============================] - 2s 3ms/step - loss: 0.0710 - accuracy: 0.9776
Epoch 4/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0593 - accuracy: 0.9813
Epoch 5/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0496 - accuracy: 0.9840
Epoch 6/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0440 - accuracy: 0.9859
Epoch 7/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0419 - accuracy: 0.9873
Epoch 8/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0449 - accuracy: 0.9868
Epoch 9/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0363 - accuracy: 0.9894
Epoch 10/10
600/600 [==============================] - 2s 4ms/step - loss: 0.0364 - accuracy: 0.9892


In [7]:
# print model layers
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (100, 784)                0         
                                                                 
 dense_2 (Dense)             (100, 200)                157000    
                                                                 
 dense_3 (Dense)             (100, 10)                 2010      
                                                                 
Total params: 159,010
Trainable params: 159,010
Non-trainable params: 0
_________________________________________________________________


### Evaluate Loss and Accuracy on Test Set

In [8]:
val_loss, val_acc = model.evaluate(x_test,y_test)
print("loss: ",val_loss,"\n","accuracy: ",val_acc)

 33/313 [==>...........................] - ETA: 0s - loss: 0.2507 - accuracy: 0.9659   

2022-07-22 16:26:34.659780: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 3ms/step - loss: 0.2082 - accuracy: 0.9672
loss:  0.20820000767707825 
 accuracy:  0.9672000408172607


In [9]:
model.save('num_model')

INFO:tensorflow:Assets written to: num_model/assets


In [10]:
new_model = tf.keras.models.load_model('num_model')

In [11]:
print(np.shape(x_test))
predictions = new_model.predict([x_test])

(10000, 28, 28)
143/313 [============>.................] - ETA: 0s

2022-07-22 16:26:36.307619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 0s 1ms/step


In [12]:
print(len(x_test))

10000


In [13]:
for i in range(len(predictions)):
    if np.argmax(predictions[i])!=y_test[i]:
        print(np.argmax(predictions[i]))
        plt.imshow(x_test[i],cmap=plt.cm.binary)
        break

9


NameError: name 'plt' is not defined

In [16]:
print(predictions[9876])

[1.2930743e-20 1.0000000e+00 1.4422655e-20 3.5164758e-22 5.2656620e-14
 2.0460422e-15 4.8591104e-17 1.4452675e-13 2.6751034e-14 2.0754780e-25]


In [15]:
plt.imshow(x_test[9876],cmap=plt.cm.binary)

NameError: name 'plt' is not defined

In [ ]:
from NN_database import Database
import pandas as pd
db = Database()

connection = db.create_db_connection("140.233.160.216", "agironda", "phys_research1", "mnist_db")

select_all_query = "SELECT * FROM mnist"

results = db.read_query(connection,select_all_query)

from_db = []

for result in results:
  result = list(result)
  from_db.append(result)

columns = ["nn_id", "hidden_units", "layers", "training_size", "learning_rate",
            "decay_lr","dropout","dropout_size","epochs","batch_size","loss",
            "metrics","activation","convolutional","accuracy"]

df = pd.DataFrame(from_db, columns=columns)
df = df[df["layers"]==3]

display(df)

In [ ]:
import matplotlib.pyplot as plt
x_var = "hidden_units"
plt.scatter(df[x_var],df["accuracy"])
plt.xlabel(x_var)
plt.ylabel("accuracy")
plt.show()

In [ ]:
# lr schedule callback
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * math.pow(0.6,epoch), verbose=True)


mnist = tf.keras.datasets.mnist
mnist = mnist.load_data()
(x_train,y_train),(x_test,y_test)=mnist
x_train,x_test = tf.keras.utils.normalize(x_train,axis=1),tf.keras.utils.normalize(x_test,axis=1)

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(kernel_size=3,filters=12,use_bias=False,padding='same'))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(kernel_size=6,filters=24,use_bias=False,padding='same',strides=2))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(kernel_size=6,filters=32,use_bias=False,padding='same',strides=2))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(200,use_bias=False))
model.add(tf.keras.layers.BatchNormalization(center=True,scale=False))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

optim = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

steps_per_epoch = len(x_train)//64

model.fit(x_train,y_train,epochs=10,steps_per_epoch = steps_per_epoch,callbacks=[lr_decay_callback])

In [ ]:
val_loss, val_acc = model.evaluate(x_test,y_test)
print(val_acc)

In [ ]:
model.save('num_model')

In [ ]:
new_model = tf.keras.models.load_model('num_model')

In [ ]:
predictions = new_model.predict([x_test])

In [ ]:
print(np.argmax(predictions[234]))
plt.imshow(x_test[234],cmap=plt.cm.binary)